<a href="https://colab.research.google.com/github/kimianj/Robot_Train/blob/main/arm_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybullet stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install gymnasium

In [7]:
from gymnasium import Env, spaces

In [9]:
import pybullet as p
import pybullet_data
import numpy as np
from stable_baselines3 import PPO
from gymnasium import Env, spaces #changed import
from stable_baselines3.common.vec_env import DummyVecEnv

class RoboticArmEnv(Env): #changed the parent class

    def __init__(self):
        super(RoboticArmEnv, self).__init__()
        self.action_space = spaces.Box(low=-np.pi, high=np.pi, shape=(num_joints,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(get_observation()),), dtype=np.float32)

    def reset(self, seed=None, options=None): #added seed and options because of gymnasium.
        observation = reset_env()
        info = {}
        return observation, info

    def step(self, action):
        observation, reward, done, _ = step(action)
        info = {}
        return observation, reward, done, False, info #stable baselines 3 expects 5 values, truncated = false.

    def render(self):
        pass #p.GUI handles rendering.

In [11]:
#reseting the postion of the joints
def reset_env():
  target_pos = [np.random.uniform(0.5, 1.5), np.random.uniform(-1, 1), np.random.uniform(0.1, 1)]
  p.resetBasePositionAndOrientation(robot_id, target_pos, [0, 0, 0, 1])

  #recalling joints and positions
  for joint_id in joint_indices:
    p.resetJointState(robot_id, jointid, 0)
  return get_observation()

In [12]:
#observing the position that is going to be used by the RL agent as input (gets the postion of the end of the arm)
def get_observation():
  joint_states = p.getJointStates(robot_id, joint_indices)
  joint_positions = [state[0] for state in joint_states]
  end_effector_pos, _ = p.getLinkState(robot_id, end_effector_index)[:2] #3D
  target_pos, _ = p.getBasePositionAndOrientation(robot_id)[:2]
  observ = np.array(joint_positions + list(end_effector_pos) + list(target_pos) + list(target_pos))
  return observ

In [13]:
#reward system that rewards agent when the action is correct
def reward():
  end_effector_pos, _ = p.getLinkState(robot_id, end_effector_index)[:2]
  target_pos, _ = p.getVasePositionAndOrientation(target_id)[:2]
  distance = np.linalg.norm(np.array(end_effector_pos) - np.array(target_pos))
  reward = -distance
  return reward

In [14]:
#checks for the postion. Acceptable if distance is very close
def is_done():
  end_effector_pos, _ = p.getLinkState(robot_id, end_effector_index)[:2]
  target_pos, _ = p.getBasePositionAndOrientation(target_id)[:2]
  distance = np.linalg.norm(np.array(end_effector_pos) - np.array(target_pos))
  return distance < 0.1

In [15]:
# if all correct, simulate the one step based on the enviroenment--> Macrkov Descision Progress
def step():
  for i, join_index in enumerate(joint_indices):
    p.setJointMotorControl2(robot_id, join_index, p.POSITION_CONTROL, targetPositions=action[i])
  p.stepSimulation()
  observ = get_observation()
  reward = reward()
  done = is_done()

  return observ, reward, done, {}